In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from collections import Counter
from math import log2

In [2]:
# Function to compute Gini Impurity
def gini_impurity(y):
    # Get the proportion of each class
    proportions = np.bincount(y) / len(y)
    # Calculate Gini
    gini = 1 - np.sum([p**2 for p in proportions if p > 0])
    return gini

In [3]:
# Function to compute Entropy
def entropy(y):
    # Get the proportion of each class
    proportions = np.bincount(y) / len(y)
    # Calculate entropy
    ent = -np.sum([p * log2(p) for p in proportions if p > 0])
    return ent

In [4]:
dataset = "/content/drive/MyDrive/Project II (Sem VII)/Liver Patient Dataset (LPD)_train.csv"

In [5]:
data = pd.read_csv(dataset, encoding='ISO-8859-1')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30691 entries, 0 to 30690
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Age of the patient                    30689 non-null  float64
 1   Gender of the patient                 29789 non-null  object 
 2   Total Bilirubin                       30043 non-null  float64
 3   Direct Bilirubin                      30130 non-null  float64
 4    Alkphos Alkaline Phosphotase         29895 non-null  float64
 5    Sgpt Alamine Aminotransferase        30153 non-null  float64
 6   Sgot Aspartate Aminotransferase       30229 non-null  float64
 7   Total Protiens                        30228 non-null  float64
 8    ALB Albumin                          30197 non-null  float64
 9   A/G Ratio Albumin and Globulin Ratio  30132 non-null  float64
 10  Result                                30691 non-null  int64  
dtypes: float64(9), 

In [7]:
data.drop('Gender of the patient', axis=1, inplace=True)

In [8]:
data = data.dropna()

In [9]:
data

Age of the patient  Total Bilirubin  Direct Bilirubin  \
0                    65.0              0.7               0.1   
1                    62.0             10.9               5.5   
2                    62.0              7.3               4.1   
3                    58.0              1.0               0.4   
4                    72.0              3.9               2.0   
...                   ...              ...               ...   
30686                50.0              2.2               1.0   
30687                55.0              2.9               1.3   
30688                54.0              6.8               3.0   
30689                48.0              1.9               1.0   
30690                30.0              3.1               1.6   

        Alkphos Alkaline Phosphotase   Sgpt Alamine Aminotransferase  \
0                              187.0                            16.0   
1                              699.0                            64.0   
2                              490.0                            60.0   
3                              182.0                            14.0   
4                              195.0                            27.0   
...                              ...                             ...   
30686                          610.0                            17.0   
30687                          482.0                            22.0   
30688                          542.0                           116.0   
30689                          231.0                            16.0   
30690                          253.0                            80.0   

       Sgot Aspartate Aminotransferase  Total Protiens   ALB Albumin  \
0                                 18.0             6.8           3.3   
1                                100.0             7.5           3.2   
2                                 68.0             7.0           3.3   
3                                 20.0             6.8           3.4   
4                                 59.0             7.3           2.4   
...                                ...             ...           ...   
30686                             28.0             7.3           2.6   
30687                             34.0             7.0           2.4   
30688                             66.0             6.4           3.1   
30689                             55.0             4.3           1.6   
30690                            406.0             6.8           3.9   

       A/G Ratio Albumin and Globulin Ratio  Result  
0                                      0.90       1  
1                                      0.74       1  
2                                      0.89       1  
3                                      1.00       1  
4                                      0.40       1  
...                                     ...     ...  
30686                                  0.55       1  
30687                                  0.50       1  
30688                                  0.90       1  
30689                                  0.60       1  
30690                                  1.30       1  

[27666 rows x 10 columns]

In [10]:
data.drop_duplicates()

Age of the patient  Total Bilirubin  Direct Bilirubin  \
0                    65.0              0.7               0.1   
1                    62.0             10.9               5.5   
2                    62.0              7.3               4.1   
3                    58.0              1.0               0.4   
4                    72.0              3.9               2.0   
...                   ...              ...               ...   
29894                32.0              0.6               0.1   
29895                61.0             10.2               4.2   
29974                58.0              2.9               1.3   
30688                54.0              6.8               3.0   
30689                48.0              1.9               1.0   

        Alkphos Alkaline Phosphotase   Sgpt Alamine Aminotransferase  \
0                              187.0                            16.0   
1                              699.0                            64.0   
2                              490.0                            60.0   
3                              182.0                            14.0   
4                              195.0                            27.0   
...                              ...                             ...   
29894                          196.0                            29.0   
29895                          232.0                            58.0   
29974                          482.0                            22.0   
30688                          542.0                           116.0   
30689                          231.0                            16.0   

       Sgot Aspartate Aminotransferase  Total Protiens   ALB Albumin  \
0                                 18.0             6.8           3.3   
1                                100.0             7.5           3.2   
2                                 68.0             7.0           3.3   
3                                 20.0             6.8           3.4   
4                                 59.0             7.3           2.4   
...                                ...             ...           ...   
29894                             30.0             5.8           2.9   
29895                            140.0             7.0           2.7   
29974                             34.0             7.0           2.4   
30688                             66.0             6.4           3.1   
30689                             55.0             4.3           1.6   

       A/G Ratio Albumin and Globulin Ratio  Result  
0                                      0.90       1  
1                                      0.74       1  
2                                      0.89       1  
3                                      1.00       1  
4                                      0.40       1  
...                                     ...     ...  
29894                                  1.00       1  
29895                                  0.60       1  
29974                                  0.50       1  
30688                                  0.90       1  
30689                                  0.60       1  

[14118 rows x 10 columns]

In [11]:
result_mapping = { 1 : 0, 2:1}

In [12]:
data['Result'] = data['Result'].map(result_mapping)

In [13]:
X = data.drop('Result', axis=1)  # All columns except the target
y = data['Result']  # Target column

In [14]:
y.isna().sum()

0

In [15]:
gini_before = gini_impurity(y)
entropy_before = entropy(y)

In [16]:
print(f"Gini Impurity before balancing: {gini_before}")
print(f"Entropy before balancing: {entropy_before}")

Gini Impurity before balancing: 0.4059324676956808
Entropy before balancing: 0.8596770667997339


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [19]:
feature_importances = pd.Series(rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print("Feature Importances:\n", feature_importances)

Feature Importances:
  Alkphos Alkaline Phosphotase           0.174698
 Sgpt Alamine Aminotransferase          0.149223
Sgot Aspartate Aminotransferase         0.148245
Total Bilirubin                         0.132609
Total Protiens                          0.107845
 ALB Albumin                            0.107112
A/G Ratio Albumin and Globulin Ratio    0.092296
Direct Bilirubin                        0.087436
Age of the patient                      0.000537
dtype: float64


In [20]:
class_counts = Counter(y)
min_class = min(class_counts, key=class_counts.get)

In [21]:
df_minority = data[data['Result'] == min_class]
df_majority = data[data['Result'] != min_class]

In [22]:
importances = rf.feature_importances_

In [23]:
importances

array([0.00053661, 0.13260893, 0.08743572, 0.17469799, 0.14922284,
       0.14824523, 0.10784501, 0.1071117 , 0.09229598])

In [24]:
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

In [25]:
top_feature = feature_importance_df.iloc[0]['Feature']
df_majority_sorted = df_majority.sort_values(by=top_feature, ascending=False)

In [26]:
df_majority_downsampled = df_majority_sorted.head(len(df_minority))

In [27]:
df_majority_downsampled

Age of the patient  Total Bilirubin  Direct Bilirubin  \
7812                 54.0              2.0               1.4   
3148                 58.0              2.0               1.4   
28864                46.0              2.0               1.4   
10144                55.0              2.0               1.4   
25553                36.0              2.0               1.4   
...                   ...              ...               ...   
19861                60.0              8.9               4.5   
8937                 37.0              2.3               0.6   
14359                51.0              1.8               9.0   
24791                36.0              8.9               4.5   
4194                 12.0              0.6               0.1   

        Alkphos Alkaline Phosphotase   Sgpt Alamine Aminotransferase  \
7812                          2110.0                            48.0   
3148                          2110.0                            48.0   
28864                         2110.0                            48.0   
10144                         2110.0                            48.0   
25553                         2110.0                            48.0   
...                              ...                             ...   
19861                          272.0                            31.0   
8937                           272.0                            79.0   
14359                          272.0                            22.0   
24791                          272.0                            31.0   
4194                           272.0                            24.0   

       Sgot Aspartate Aminotransferase  Total Protiens   ALB Albumin  \
7812                              89.0             6.2           3.0   
3148                              89.0             6.2           3.0   
28864                             89.0             6.2           3.0   
10144                             89.0             6.2           3.0   
25553                             89.0             6.2           3.0   
...                                ...             ...           ...   
19861                             61.0             5.8           2.0   
8937                              51.0             6.6           3.5   
14359                             79.0             6.1           2.7   
24791                             61.0             5.8           2.0   
4194                              98.0             5.0           2.0   

       A/G Ratio Albumin and Globulin Ratio  Result  
7812                                    0.9       0  
3148                                    0.9       0  
28864                                   0.9       0  
10144                                   0.9       0  
25553                                   0.9       0  
...                                     ...     ...  
19861                                   0.5       0  
8937                                    1.1       0  
14359                                   0.7       0  
24791                                   0.5       0  
4194                                    0.6       0  

[7833 rows x 10 columns]

In [28]:
balanced_data = pd.concat([df_minority, df_majority_downsampled])

In [29]:
y_balanced = balanced_data['Result']
gini_after = gini_impurity(y_balanced)
entropy_after = entropy(y_balanced)
print(f"Gini Impurity after balancing: {gini_after}")
print(f"Entropy after balancing: {entropy_after}")

Gini Impurity after balancing: 0.5
Entropy after balancing: 1.0


In [30]:
X_balanced = balanced_data.drop('Result', axis=1)  # Replace 'Dataset' with your target column name
y_balanced = balanced_data['Result']
X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

In [31]:
rf_bal = RandomForestClassifier(random_state=42)
rf_bal.fit(X_train_bal, y_train_bal)
y_pred_rf_bal = rf_bal.predict(X_test_bal)

In [32]:
xgb = XGBClassifier(random_state=42)
xgb.fit(X_train_bal, y_train_bal)
y_pred_xgb = xgb.predict(X_test_bal)

In [33]:
print("Random Forest Accuracy:", accuracy_score(y_test_bal, y_pred_rf_bal))
print("XGBoost Accuracy:", accuracy_score(y_test_bal, y_pred_xgb))
print("\nClassification Report for Random Forest:\n", classification_report(y_test_bal, y_pred_rf_bal))
print("\nClassification Report for XGBoost:\n", classification_report(y_test_bal, y_pred_xgb))

Random Forest Accuracy: 0.9996809189534142
XGBoost Accuracy: 0.9996809189534142

Classification Report for Random Forest:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1569
           1       1.00      1.00      1.00      1565

    accuracy                           1.00      3134
   macro avg       1.00      1.00      1.00      3134
weighted avg       1.00      1.00      1.00      3134


Classification Report for XGBoost:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1569
           1       1.00      1.00      1.00      1565

    accuracy                           1.00      3134
   macro avg       1.00      1.00      1.00      3134
weighted avg       1.00      1.00      1.00      3134



In [34]:
comparison_df = X_test_bal.copy()
comparison_df['Actual'] = y_test_bal
comparison_df['Predicted'] = y_pred_rf_bal
# df = pd.DataFrame({'inputs' : X_train, 'Actual': y_test_bal, 'Predicted': y_pred_rf_bal })

In [35]:
wrong_predictions = comparison_df[comparison_df['Actual'] != comparison_df['Predicted']]

In [36]:
wrong_predictions

Age of the patient  Total Bilirubin  Direct Bilirubin  \
9162                45.0              2.2               1.0   

       Alkphos Alkaline Phosphotase   Sgpt Alamine Aminotransferase  \
9162                          610.0                            17.0   

      Sgot Aspartate Aminotransferase  Total Protiens   ALB Albumin  \
9162                             28.0             7.3           2.6   

      A/G Ratio Albumin and Globulin Ratio  Actual  Predicted  
9162                                  0.55       1          0

In [37]:
comparison_df['Predicted'] = y_pred_xgb

In [38]:
wrong_predictions = comparison_df[comparison_df['Actual'] != comparison_df['Predicted']]

In [39]:
wrong_predictions

Age of the patient  Total Bilirubin  Direct Bilirubin  \
9162                45.0              2.2               1.0   

       Alkphos Alkaline Phosphotase   Sgpt Alamine Aminotransferase  \
9162                          610.0                            17.0   

      Sgot Aspartate Aminotransferase  Total Protiens   ALB Albumin  \
9162                             28.0             7.3           2.6   

      A/G Ratio Albumin and Globulin Ratio  Actual  Predicted  
9162                                  0.55       1          0

In [40]:
print(xgb.score(X_train_bal,y_train_bal))
xgb.score(X_test_bal,y_test_bal)

0.9999202042770508


0.9996809189534142

In [41]:
print(rf.score(X_train_bal,y_train_bal))
rf.score(X_test_bal,y_test_bal)

0.9997606128311523


0.9996809189534142

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search over
param_grid = {
    'n_estimators': [4,7,9],         # Number of trees in the forest
    'max_depth': [10, 15],        # Maximum depth of the tree
    'min_samples_split': [ 5, 10],
 # Minimum number of samples required to split a node
    'min_samples_leaf': [ 4,7,9],          # Minimum number of samples required at a leaf node        # Whether bootstrap samples are used when building trees
}


In [43]:
rf_bal = RandomForestClassifier(random_state=42)

# Set up GridSearchCV with cross-validation
grid_search = GridSearchCV(estimator=rf_bal, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(X_train_bal, y_train_bal)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Print out the best parameters and the best score
print("Best Parameters:", best_params)
print("Best Cross-Validation Score:", best_score)

# Use the best model for prediction
best_rf_model = grid_search.best_estimator_
y_pred_rf_bal = best_rf_model.predict(X_test_bal)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 7}
Best Cross-Validation Score: 0.9999201979091853
